# Generates images from text prompts with VQGAN and CLIP (z+quantize method) + editor masks.

By https://twitter.com/jbusted1 .
Based on a notebook by Katherine Crowson (https://github.com/crowsonkb, https://twitter.com/RiversHaveWings)


In [ ]:
#@markdown Your GPU (p100 or v100 are preferable)
!nvidia-smi

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
#@markdown #install relevant libraries
!git clone https://github.com/openai/CLIP
!git clone https://github.com/CompVis/taming-transformers
!pip install ftfy regex tqdm omegaconf pytorch-lightning
!pip install kornia
!pip install einops

In [ ]:
#@markdown #download vqgan weights

!curl -L 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fconfigs%2Fmodel.yaml&dl=1' > vqgan_imagenet_f16_16384.yaml
!curl -L 'https://heibox.uni-heidelberg.de/d/a7530b09fed84f80a887/files/?p=%2Fckpts%2Flast.ckpt&dl=1' > vqgan_imagenet_f16_16384.ckpt

# !curl -L https://dl.nmkd.de/ai/clip/vqgan/8k-2021-06/vqgan-f8-8192.ckpt > vqgan_openimages_f16_8192.ckpt
# !curl -L https://dl.nmkd.de/ai/clip/vqgan/8k-2021-06/vqgan-f8-8192.yaml > vqgan_openimages_f16_8192.yaml


In [2]:
#@markdown #import libraries
import argparse
import math
from pathlib import Path
import requests
import sys
import io

sys.path.append('./taming-transformers')

from IPython import display
from omegaconf import OmegaConf
from PIL import Image
from taming.models import cond_transformer, vqgan
import torch
from torch import nn, optim
from torch.nn import functional as F
from torchvision import transforms
from torchvision.transforms import functional as TF
from tqdm.notebook import tqdm
import numpy as np

from CLIP import clip

import kornia.augmentation as K

In [3]:
#@markdown #helpful functions
def noise_gen(shape):
    n, c, h, w = shape
    noise = torch.zeros([n, c, 1, 1])
    for i in reversed(range(5)):
        h_cur, w_cur = h // 2**i, w // 2**i
        noise = F.interpolate(noise, (h_cur, w_cur), mode='bicubic', align_corners=False)
        noise += torch.randn([n, c, h_cur, w_cur]) / 5
    return noise


def sinc(x):
    return torch.where(x != 0, torch.sin(math.pi * x) / (math.pi * x), x.new_ones([]))


def lanczos(x, a):
    cond = torch.logical_and(-a < x, x < a)
    out = torch.where(cond, sinc(x) * sinc(x/a), x.new_zeros([]))
    return out / out.sum()


def ramp(ratio, width):
    n = math.ceil(width / ratio + 1)
    out = torch.empty([n])
    cur = 0
    for i in range(out.shape[0]):
        out[i] = cur
        cur += ratio
    return torch.cat([-out[1:].flip([0]), out])[1:-1]


def resample(input, size, align_corners=True):
    n, c, h, w = input.shape
    dh, dw = size

    input = input.view([n * c, 1, h, w])

    if dh < h:
        kernel_h = lanczos(ramp(dh / h, 2), 2).to(input.device, input.dtype)
        pad_h = (kernel_h.shape[0] - 1) // 2
        input = F.pad(input, (0, 0, pad_h, pad_h), 'reflect')
        input = F.conv2d(input, kernel_h[None, None, :, None])

    if dw < w:
        kernel_w = lanczos(ramp(dw / w, 2), 2).to(input.device, input.dtype)
        pad_w = (kernel_w.shape[0] - 1) // 2
        input = F.pad(input, (pad_w, pad_w, 0, 0), 'reflect')
        input = F.conv2d(input, kernel_w[None, None, None, :])

    input = input.view([n, c, h, w])
    return F.interpolate(input, size, mode='bicubic', align_corners=align_corners)
    

# def replace_grad(fake, real):
#     return fake.detach() - real.detach() + real


class ReplaceGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x_forward, x_backward):
        ctx.shape = x_backward.shape
        return x_forward

    @staticmethod
    def backward(ctx, grad_in):
        return None, grad_in.sum_to_size(ctx.shape)


class ClampWithGrad(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, min, max):
        ctx.min = min
        ctx.max = max
        ctx.save_for_backward(input)
        return input.clamp(min, max)

    @staticmethod
    def backward(ctx, grad_in):
        input, = ctx.saved_tensors
        return grad_in * (grad_in * (input - input.clamp(ctx.min, ctx.max)) >= 0), None, None

replace_grad = ReplaceGrad.apply

clamp_with_grad = ClampWithGrad.apply
# clamp_with_grad = torch.clamp

def vector_quantize(x, codebook):
    d = x.pow(2).sum(dim=-1, keepdim=True) + codebook.pow(2).sum(dim=1) - 2 * x @ codebook.T
    indices = d.argmin(-1)
    x_q = F.one_hot(indices, codebook.shape[0]).to(d.dtype) @ codebook
    return replace_grad(x_q, x)


class Prompt(nn.Module):
    def __init__(self, embed, weight=1., stop=float('-inf')):
        super().__init__()
        self.register_buffer('embed', embed)
        self.register_buffer('weight', torch.as_tensor(weight))
        self.register_buffer('stop', torch.as_tensor(stop))

    def forward(self, input):
        
        input_normed = F.normalize(input.unsqueeze(1), dim=2)
        embed_normed = F.normalize((self.embed).unsqueeze(0), dim=2)

        dists = input_normed.sub(embed_normed).norm(dim=2).div(2).arcsin().pow(2).mul(2)
        dists = dists * self.weight.sign()
        return self.weight.abs() * replace_grad(dists, torch.maximum(dists, self.stop)).mean()


def parse_prompt(prompt):
    vals = prompt.rsplit(':', 2)
    vals = vals + ['', '1', '-inf'][len(vals):]
    return vals[0], float(vals[1]), float(vals[2])

def one_sided_clip_loss(input, target, labels=None, logit_scale=100):
    input_normed = F.normalize(input, dim=-1)
    target_normed = F.normalize(target, dim=-1)
    logits = input_normed @ target_normed.T * logit_scale
    if labels is None:
        labels = torch.arange(len(input), device=logits.device)
    return F.cross_entropy(logits, labels)

class MakeCutouts(nn.Module):
    def __init__(self, cut_size, cutn, cut_pow=1.):
        super().__init__()
        self.cut_size = cut_size
        self.cutn = cutn
        self.cut_pow = cut_pow


    def set_cut_pow(self, cut_pow):
      self.cut_pow = cut_pow

    def forward(self, input):
        sideY, sideX = input.shape[2:4]
        max_size = min(sideX, sideY)
        min_size = min(sideX, sideY, self.cut_size)
        cutouts = []
        cutouts_full = []
        
        min_size_width = min(sideX, sideY)
        lower_bound = float(self.cut_size/min_size_width)
        
        for ii in range(self.cutn):
            
          if args.clip_model == 'ViT-B/16':
            size = int(min_size_width*torch.zeros(1,).normal_(mean=.8, std=.3).clip(lower_bound, 1.))
          else:
            # size = int(torch.rand([])**self.cut_pow * (max_size - min_size) + min_size)
            size = int(min_size_width*torch.zeros(1,).normal_(mean=.8, std=.3).clip(lower_bound, 1.)) # replace .5 with a result for 224 the default large size is .95
            ## size = int(min_size_width*torch.zeros(1,).normal_(mean=.9, std=.3).clip(lower_bound, .95)) # replace .5 with a result for 224 the default large size is .95

          offsetx = torch.randint(0, sideX - size + 1, ())
          offsety = torch.randint(0, sideY - size + 1, ())
          cutout = input[:, :, offsety:offsety + size, offsetx:offsetx + size]
          cutouts.append(resample(cutout, (self.cut_size, self.cut_size)))

        
        cutouts = torch.cat(cutouts, dim=0)

        # if args.use_augs:
        #   cutouts = augs(cutouts)

        # if args.noise_fac:
        #   facs = cutouts.new_empty([cutouts.shape[0], 1, 1, 1]).uniform_(0, args.noise_fac)
        #   cutouts = cutouts + facs * torch.randn_like(cutouts)
        

        return clamp_with_grad(cutouts, 0, 1)


def load_vqgan_model(config_path, checkpoint_path):
    config = OmegaConf.load(config_path)
    if config.model.target == 'taming.models.vqgan.VQModel':
        model = vqgan.VQModel(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    elif config.model.target == 'taming.models.cond_transformer.Net2NetTransformer':
        parent_model = cond_transformer.Net2NetTransformer(**config.model.params)
        parent_model.eval().requires_grad_(False)
        parent_model.init_from_ckpt(checkpoint_path)
        model = parent_model.first_stage_model
    elif config.model.target == 'taming.models.vqgan.GumbelVQ':
        model = vqgan.GumbelVQ(**config.model.params)
        model.eval().requires_grad_(False)
        model.init_from_ckpt(checkpoint_path)
    else:
        raise ValueError(f'unknown model type: {config.model.target}')
    del model.loss
    return model

def resize_image(image, out_size):
    ratio = image.size[0] / image.size[1]
    area = min(image.size[0] * image.size[1], out_size[0] * out_size[1])
    size = round((area * ratio)**0.5), round((area / ratio)**0.5)
    return image.resize(size, Image.LANCZOS)

def fetch(url_or_path):
    if str(url_or_path).startswith('http://') or str(url_or_path).startswith('https://'):
        r = requests.get(url_or_path)
        r.raise_for_status()
        fd = io.BytesIO()
        fd.write(r.content)
        fd.seek(0)
        return fd
    return open(url_or_path, 'rb')

def visulize_regions(pil_image, z_mask):

  OPACITY = int(255 * 0.5)

  pil_image = pil_image.resize(size=(z_mask.shape[1]*16, z_mask.shape[0]*16))
  overlay = Image.new('RGBA', pil_image.size, (0, 0, 0)+(0,))
  d = ImageDraw.Draw(overlay)

  ww,hh = z_mask.shape

  for x in range(0,hh):
    for y in range(0,ww):
      if z_mask[y,x] == 0:
        xx = x*16
        yy = y*16
        d.rectangle([(xx,yy),(xx+16,yy+16)],fill=(1,1,1)+(OPACITY,),width=16)

  return Image.alpha_composite(pil_image, overlay)

class TVLoss(nn.Module):
    def forward(self, input):
        input = F.pad(input, (0, 1, 0, 1), 'replicate')
        x_diff = input[..., :-1, 1:] - input[..., :-1, :-1]
        y_diff = input[..., 1:, :-1] - input[..., :-1, :-1]
        diff = x_diff**2 + y_diff**2 + 1e-8
        return diff.mean(dim=1).sqrt().mean()

class GaussianBlur2d(nn.Module):
    def __init__(self, sigma, window=0, mode='reflect', value=0):
        super().__init__()
        self.mode = mode
        self.value = value
        if not window:
            window = max(math.ceil((sigma * 6 + 1) / 2) * 2 - 1, 3)
        if sigma:
            kernel = torch.exp(-(torch.arange(window) - window // 2)**2 / 2 / sigma**2)
            kernel /= kernel.sum()
        else:
            kernel = torch.ones([1])
        self.register_buffer('kernel', kernel)

    def forward(self, input):
        n, c, h, w = input.shape
        input = input.view([n * c, 1, h, w])
        start_pad = (self.kernel.shape[0] - 1) // 2
        end_pad = self.kernel.shape[0] // 2
        input = F.pad(input, (start_pad, end_pad, start_pad, end_pad), self.mode, self.value)
        input = F.conv2d(input, self.kernel[None, None, None, :])
        input = F.conv2d(input, self.kernel[None, None, :, None])
        return input.view([n, c, h, w])

class EMATensor(nn.Module):
    """implmeneted by Katherine Crowson"""
    def __init__(self, tensor, decay):
        super().__init__()
        self.tensor = nn.Parameter(tensor)
        self.register_buffer('biased', torch.zeros_like(tensor))
        self.register_buffer('average', torch.zeros_like(tensor))
        self.decay = decay
        self.register_buffer('accum', torch.tensor(1.))
        self.update()
    
    @torch.no_grad()
    def update(self):
        if not self.training:
            raise RuntimeError('update() should only be called during training')

        self.accum *= self.decay
        self.biased.mul_(self.decay)
        self.biased.add_((1 - self.decay) * self.tensor)
        self.average.copy_(self.biased)
        self.average.div_(1 - self.accum)

    def forward(self):
        if self.training:
            return self.tensor
        return self.average

%mkdir /content/vids

# ARGS VIT B32

In [6]:

args = argparse.Namespace(
    
    prompts=[["a photo of the stary dark sky at night."],["a photo of a gothic palace at the street"]], 
    size=[640, 512], 
    init_image= "https://www.themontcalm.com/blog/wp-content/uploads/2018/04/piccadilly.jpg", 
    init_weight=None,

    clip_model='ViT-B/32',
    vqgan_config='vqgan_imagenet_f16_16384.yaml',        
    vqgan_checkpoint='vqgan_imagenet_f16_16384.ckpt', 
    step_size=0.25, 
    
    cutn=50, 
    cut_pow=1, 
    
    display_freq=25,
    seed=1587585,
    use_augs = True,
    noise_fac= 0.1,
    ema_val = 0.99,
    
    record_generation = True,
    use_noise = None,
    constraint_regions = False, 
    
    noise_prompt_weights = None,
    noise_prompt_seeds = [[14575]],
   
    decay_rate = 50,
    _epoches = 5, 
)

# ---

# <AUGMENTATIONS>
augs = nn.Sequential(
    K.RandomHorizontalFlip(p=0.5),
    K.RandomAffine(degrees=30, translate=0.1, p=0.8, padding_mode='border'), # padding_mode=2
    K.RandomPerspective(0.2,p=0.4, ),
    K.ColorJitter(hue=0.01, saturation=0.01, p=0.7),
    )

noise = noise_gen([1, 3, args.size[0], args.size[1]])
image = TF.to_pil_image(noise.div(5).add(0.5).clamp(0, 1)[0])
image.save('init3.png')

In [ ]:
#@markdown #Paint masks
#@markdown the int input is the size of your brush
from IPython.display import HTML
from IPython.display import clear_output
# from Ipython.display import Image as Image_
from PIL import ImageDraw
import IPython.core.display as display_
from google.colab.output import eval_js
from base64 import b64decode

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

canvas_html = """
<canvas height=%d width=%d style="%s"></canvas>
<button>Finish</button>
<input type="number" id="width" name="width" size="40" value="50">
<input type="text" id="color" name="color" size="40">
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.lineCap = "round"
ctx.lineJoin = "round"
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}
canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.strokeStyle = document.getElementById("color").value
  ctx.lineWidth = document.getElementById("width").value
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

def draw(filename='drawing.png', w=640, h=512, image_path=""):

  f = 16
  if(args.clip_model == 'ViT-B/16'):
    f = 8

  display_.display(HTML(canvas_html % (h, w, image_path)))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  clear_output()
  
  # pil_image = Image.open(filename).convert('RGBA')
  pil_image = Image.open(io.BytesIO(binary)).convert('RGBA')
  toksX, toksY = w // f, h // f
  pil_image = TF.to_tensor(pil_image)[3].unsqueeze(0).unsqueeze(0)

  return F.interpolate(pil_image, size=(int(args.size[1]/f), int(args.size[0]/f)), mode='nearest').squeeze()

def get_z_mask():
  image_path = ""

  if args.init_image is not None:
    image_path = f"background: url('{args.init_image}')"
    pil_image = Image.open(fetch(args.init_image)).convert('RGBA')
    w,h = pil_image.size
    z_mask = draw(filename='drawing.png', w=w, h=h, image_path=image_path)
  else:
    
    pil_image = None 
    image_path = "border:1px solid #000000;"
    z_mask = draw(filename='drawing.png', image_path=image_path)
  
  return z_mask, pil_image

z_masks = []
for _ind in range(len(args.prompts)):
  print(args.prompts[_ind])
  z_mask, pil_image_ = get_z_mask()
  z_masks.append(z_mask)
  
  

for _ind in range(len(z_masks) - 1):
  z_masks[_ind] = torch.max(z_masks[_ind] - sum(z_masks[_ind + 1:]), torch.zeros_like(z_masks[_ind]))


for _ind in range(len(z_masks)):
  if pil_image_ is None:
    pil_image_ = Image.new("RGBA", (800, 1280), (255, 255, 255))
  print(args.prompts[_ind])
  h = visulize_regions(pil_image_, z_masks[_ind])

    

  h.save(f'maskvis_{_ind}.png')
  display.display(display.Image(f'maskvis_{_ind}.png'))


  z_masks[_ind] = z_masks[_ind].unsqueeze(0).unsqueeze(0).repeat(1,256,1,1).to(device)
  

  

# https://i.imgur.com/7k0YQWK.png

# Actually do the run...

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print('Using device:', device)
print('using prompts: ', args.prompts)

tv_loss = TVLoss() 

model = load_vqgan_model(args.vqgan_config, args.vqgan_checkpoint).to(device)
perceptor = clip.load(args.clip_model, jit=False)[0].eval().requires_grad_(False).to(device)
mse_weight = args.init_weight

cut_size = perceptor.visual.input_resolution
# e_dim = model.quantize.e_dim

if args.vqgan_checkpoint == 'vqgan_openimages_f16_8192.ckpt':
    e_dim = 256
    n_toks = model.quantize.n_embed
    z_min = model.quantize.embed.weight.min(dim=0).values[None, :, None, None]
    z_max = model.quantize.embed.weight.max(dim=0).values[None, :, None, None]
else:
    e_dim = model.quantize.e_dim
    n_toks = model.quantize.n_e
    z_min = model.quantize.embedding.weight.min(dim=0).values[None, :, None, None]
    z_max = model.quantize.embedding.weight.max(dim=0).values[None, :, None, None]


make_cutouts = MakeCutouts(cut_size, args.cutn, cut_pow=args.cut_pow)

f = 2**(model.decoder.num_resolutions - 1)
toksX, toksY = args.size[0] // f, args.size[1] // f

if args.seed is not None:
    torch.manual_seed(args.seed)

if args.init_image:
    pil_image = Image.open(fetch(args.init_image)).convert('RGB')
    pil_image = pil_image.resize((toksX * f, toksY * f), Image.LANCZOS)
    pil_image = TF.to_tensor(pil_image)
    if args.use_noise:
      pil_image = pil_image + args.use_noise * torch.randn_like(pil_image) 
    z, *_ = model.encode(pil_image.to(device).unsqueeze(0) * 2 - 1)

else:
    
    one_hot = F.one_hot(torch.randint(n_toks, [toksY * toksX], device=device), n_toks).float()

    if args.vqgan_checkpoint == 'vqgan_openimages_f16_8192.ckpt':
        z = one_hot @ model.quantize.embed.weight
    else:
        z = one_hot @ model.quantize.embedding.weight
    z = z.view([-1, toksY, toksX, e_dim]).permute(0, 3, 1, 2)


z = EMATensor(z, args.ema_val)

z_orig = z.tensor.clone()


opt = optim.Adam(z.parameters(), lr=args.step_size, weight_decay=0.00000000)

normalize = transforms.Normalize(mean=[0.48145466, 0.4578275, 0.40821073],
                                 std=[0.26862954, 0.26130258, 0.27577711])
pMss = []
for p_index in range(len(args.prompts)):
  pMs = []

  if args.noise_prompt_weights and args.noise_prompt_seeds:
    for seed, weight in zip(args.noise_prompt_seeds, args.noise_prompt_weights):
      gen = torch.Generator().manual_seed(seed)
      embed = torch.empty([1, perceptor.visual.output_dim]).normal_(generator=gen)
      pMs.append(Prompt(embed, weight).to(device))

  for prompt in args.prompts[p_index]:
      txt, weight, stop = parse_prompt(prompt)
      embed = perceptor.encode_text(clip.tokenize(txt).to(device)).float()
      pMs.append(Prompt(embed, weight, stop).to(device))
      # pMs[0].embed = pMs[0].embed + Prompt(embed, weight, stop).embed.to(device)

  pMss.append(pMs)

def synth(z, quantize=True):
    # if args.constraint_regions:
    #   z = replace_grad(z, z * z_mask)

    if quantize:
      if args.vqgan_checkpoint == 'vqgan_openimages_f16_8192.ckpt':
        z_q = vector_quantize(z.movedim(1, 3), model.quantize.embed.weight).movedim(3, 1)
      else:
        z_q = vector_quantize(z.movedim(1, 3), model.quantize.embedding.weight).movedim(3, 1)

    else:
      z_q = z.model

    return clamp_with_grad(model.decode(z_q).add(1).div(2), 0, 1)

@torch.no_grad()
def checkin(i, losses):
    losses_str = ', '.join(f'{loss.item():g}' for loss in losses)
    tqdm.write(f'i: {i}, loss: {sum(losses).item():g}, losses: {losses_str}')

    print("averaged:")
    out = synth(z.average, True)
    TF.to_pil_image(out[0].cpu()).save('progress.png')   
    display.display(display.Image('progress.png')) 

def get_encodings():
    
    out = synth(z.tensor)

    if args.record_generation:
      with torch.no_grad():
        global vid_index
        out_a = synth(z.average, True)
        TF.to_pil_image(out_a[0].cpu()).save(f'/content/vids/{vid_index}.png')
        vid_index += 1
    
    cutouts = make_cutouts(out)

    if args.use_augs:
      cutouts = augs(cutouts)

    if args.noise_fac:
      facs = cutouts.new_empty([args.cutn, 1, 1, 1]).uniform_(0, args.noise_fac)
      cutouts = cutouts + facs * torch.randn_like(cutouts)

    return perceptor.encode_image(normalize(cutouts)).float()



def ascend_txt(iii, m_i, pMss):
    global z_orig   

    result = []
    if args.init_weight:
      result.append(F.mse_loss(z.tensor, z_orig) * mse_weight / 2)        

    for prompt in pMss:
        result.append(prompt(iii))

    return result

vid_index = 0
def train(i):
    
    opt.zero_grad()

    
    iii = get_encodings()
    for m_i in range(len(args.prompts)):
      
      h = z.tensor.register_hook(lambda grad: grad * z_masks[m_i])

      lossAll = ascend_txt(iii, m_i, pMss[m_i])
      loss = sum(lossAll)
      loss.backward(retain_graph=True)

      h.remove()

    opt.step()
    z.update()

    if i % args.display_freq == 0:
          checkin(i, lossAll)

    # if args.init_weight and args.use_mse_sch:
    #   update_mse()

i = 0
try:
    
    with tqdm() as pbar:
        while True and i < 3000:           

            train(i)

            if i > 0 and i%args.decay_rate==0 and i <= args.decay_rate * args._epoches:
              z = EMATensor(z.average, args.ema_val)
              opt = optim.Adam(z.parameters(), lr=args.step_size, weight_decay=0.00000000) 
              print("restarting optimization")
            i += 1
            pbar.update()

except KeyboardInterrupt:
    pass


# create video

In [ ]:
%cd vids

images = "%d.png"
video = "/content/monster2.mp4"
!ffmpeg -r 60 -i $images -crf 20 -s 640x512 -pix_fmt yuv420p $video

%cd ..

delete all frames from folder

In [ ]:
%cd vids
%rm *.png
%cd ..